<a href="https://colab.research.google.com/github/yasheymateen/Deep_Learning_MRIs/blob/master/MRI_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade --quiet mdai

     |████████████████████████████████| 35.3MB 80kB/s 
     |████████████████████████████████| 51kB 8.6MB/s 


In [0]:
import mdai
import pandas as pd
from pathlib import Path

In [0]:
DOMAIN = 'public.md.ai'
ACCESS_TOKEN = '004e60b14e612766896ae17a6d0100a5'
project_id = 'XQqbVvqr'
train_dataset_id = 'D_7E0l4E'
test_dataset_id = 'D_zQ1lMo'
mdai_client = mdai.Client(domain=DOMAIN, access_token=ACCESS_TOKEN)

Successfully authenticated to public.md.ai.


In [0]:
p = mdai_client.project(project_id, annotations_only=True)

Using working directory for data.
Preparing annotations export for project XQqbVvqr...                                                


21.3MB [00:00, 191MB/s]                            

Success: annotations data for project XQqbVvqr ready.
No project created. Downloaded annotations only.


In [0]:
paths = [str(p) for p in Path.cwd().glob("**/*.json")]
JSON = paths[0]
results = mdai.common_utils.json_to_dataframe(JSON)
annotations = results['annotations']
annotations.head()

,id,createdAt,createdById,updatedAt,updatedById,modelId,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,labelId,annotationNumber,height,width,data,note,radlexTagIds,reviewsPositiveCount,reviewsNegativeCount,dataset,groupId,groupName,annotationMode,color,description,labelName,radlexTagIdsLabel,scope,number
0,A_JddPQE,2020-02-18T21:13:01.175Z,U_w8RBv8,2020-02-18T21:13:01.175Z,U_w8RBv8,None,1.3.46.670589.11.20182.5.0.8336.20130724083604...,1.3.46.670589.11.20182.5.0.7188.20130724085044...,1.3.6.1.4.1.9590.100.1.2.260616927513309992437...,L_8W6Ord,1,256,256,"{'vertices': [[64, 130], [64, 129], [64, 128],...",None,[],0,0,Train,G_MKGlRd,CHAOS,freeform,#4caf50,,Liver,[],INSTANCE,1
1,A_qG28ZY,2020-02-18T21:13:01.175Z,U_w8RBv8,2020-02-18T21:13:01.175Z,U_w8RBv8,None,1.3.46.670589.11.20182.5.0.8336.20130724083604...,1.3.46.670589.11.20182.5.0.7188.20130724085044...,1.3.6.1.4.1.9590.100.1.2.388757691711551811514...,L_8W6Ord,1,256,256,"{'vertices': [[66, 129], [66, 128], [66, 127],...",None,[],0,0,Train,G_MKGlRd,CHAOS,freeform,#4caf50,,Liver,[],INSTANCE,1
2,A_47VBpv,2020-02-18T21:13:01.175Z,U_w8RBv8,2020-02-18T21:13:01.175Z,U_w8RBv8,None,1.3.46.670589.11.20182.5.0.8336.20130724083604...,1.3.46.670589.11.20182.5.0.7188.20130724085044...,1.3.6.1.4.1.9590.100.1.2.355688831611728149411...,L_8W6Ord,1,256,256,"{'vertices': [[69, 133], [69, 132], [69, 131],...",None,[],0,0,Train,G_MKGlRd,CHAOS,freeform,#4caf50,,Liver,[],INSTANCE,1
3,A_JAnGdr,2020-02-18T21:13:01.175Z,U_w8RBv8,2020-02-18T21:13:01.175Z,U_w8RBv8,None,1.3.46.670589.11.20182.5.0.8336.20130724083604...,1.3.46.670589.11.20182.5.0.7188.20130724085044...,1.3.6.1.4.1.9590.100.1.2.417210802510957152442...,L_8W6Ord,1,256,256,"{'vertices': [[74, 126], [74, 125], [74, 124],...",None,[],0,0,Train,G_MKGlRd,CHAOS,freeform,#4caf50,,Liver,[],INSTANCE,1
4,A_JN8WKy,2020-02-18T21:13:01.175Z,U_w8RBv8,2020-02-18T21:13:01.175Z,U_w8RBv8,None,1.3.46.670589.11.20182.5.0.8336.20130724083604...,1.3.46.670589.11.20182.5.0.7188.20130724085044...,1.3.6.1.4.1.9590.100.1.2.284053595913909049118...,L_8W6Ord,1,256,256,"{'vertices': [[78, 126], [78, 125], [78, 124],...",None,[],0,0,Train,G_MKGlRd,CHAOS,freeform,#4caf50,,Liver,[],INSTANCE,1


In [0]:
annotations.columns

Index(['id', 'createdAt', 'createdById', 'updatedAt', 'updatedById', 'modelId',
       'StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID', 'labelId',
       'annotationNumber', 'height', 'width', 'data', 'note', 'radlexTagIds',
       'reviewsPositiveCount', 'reviewsNegativeCount', 'dataset', 'groupId',
       'groupName', 'annotationMode', 'color', 'description', 'labelName',
       'radlexTagIdsLabel', 'scope', 'number'],
      dtype='object')

In [0]:
annotations.dataset.value_counts()

Train    6511
Test     4074
Name: dataset, dtype: int64

In [0]:
studies = results['studies']
studies.head()

,StudyInstanceUID,dataset,number
0,1.3.46.670589.11.20182.5.0.8336.20130724083604...,Train,1
1,1.2.752.24.7.2066097694.314180,Train,2
2,1.2.752.24.7.2066097694.320548,Train,3
3,1.2.752.24.7.2066097694.323785,Train,4
4,1.2.752.24.7.595070831.4907659,Train,5


In [0]:
# boxing data into separate columns
a = annotations[annotations.annotationMode == 'bbox'].copy()

a['x'] = [d['x'] for _,d in a.data.iteritems()]
a['y'] = [d['y'] for _,d in a.data.iteritems()]
a['w'] = [d['width'] for _,d in a.data.iteritems()]
a['h'] = [d['height'] for _,d in a.data.iteritems()]
a['x2'] = a.x + a.w
a['y2'] = a.y + a.h

a.head()

,id,createdAt,createdById,updatedAt,updatedById,modelId,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,labelId,annotationNumber,height,width,data,note,radlexTagIds,reviewsPositiveCount,reviewsNegativeCount,dataset,groupId,groupName,annotationMode,color,description,labelName,radlexTagIdsLabel,scope,number,x,y,w,h,x2,y2
2587,A_46VVeA,2020-02-20T22:56:39.070Z,U_w8RBv8,2020-02-20T22:56:42.012Z,U_w8RBv8,None,1.2.124.113532.192.70.134.138.20060119.151934....,1.3.46.670589.10.1.1.2163374719.1137684481.588647,1.3.6.1.4.1.9590.100.1.2.143996816212842704408...,L_8GwaX8,1,512,512,"{'x': 319.09049, 'y': 133.04872, 'width': 58.8...",None,[],0,0,Train,G_Zd0jg2,Stomach,bbox,#4fc3f7,,Stomach,[],INSTANCE,34,319.09049,133.04872,58.80280,132.45476,377.89329,265.50348
2588,A_4Ejj0n,2020-02-20T22:56:43.149Z,U_w8RBv8,2020-02-20T22:56:46.322Z,U_w8RBv8,None,1.2.124.113532.192.70.134.138.20060119.151934....,1.3.46.670589.10.1.1.2163374719.1137684481.588647,1.3.6.1.4.1.9590.100.1.2.221738641111322743413...,L_8GwaX8,1,512,512,"{'x': 319.09049, 'y': 133.04872, 'width': 71.8...",None,[],0,0,Train,G_Zd0jg2,Stomach,bbox,#4fc3f7,,Stomach,[],INSTANCE,34,319.09049,133.04872,71.87009,141.36428,390.96058,274.41300
2589,A_qnNN7D,2020-02-20T22:56:48.400Z,U_w8RBv8,2020-02-20T22:56:53.874Z,U_w8RBv8,None,1.2.124.113532.192.70.134.138.20060119.151934....,1.3.46.670589.10.1.1.2163374719.1137684481.588647,1.3.6.1.4.1.9590.100.1.2.349787558310949905409...,L_8GwaX8,1,512,512,"{'x': 307.80512, 'y': 127.703, 'width': 98.004...",None,[],0,0,Train,G_Zd0jg2,Stomach,bbox,#4fc3f7,,Stomach,[],INSTANCE,34,307.80512,127.70300,98.00464,147.89791,405.80976,275.60091
2590,A_J0VV8r,2020-02-20T22:56:54.923Z,U_w8RBv8,2020-02-20T22:56:59.882Z,U_w8RBv8,None,1.2.124.113532.192.70.134.138.20060119.151934....,1.3.46.670589.10.1.1.2163374719.1137684481.588647,1.3.6.1.4.1.9590.100.1.2.246608410712272726341...,L_8GwaX8,1,512,512,"{'x': 300.6775, 'y': 111.07192, 'width': 114.0...",None,[],0,0,Train,G_Zd0jg2,Stomach,bbox,#4fc3f7,,Stomach,[],INSTANCE,34,300.67750,111.07192,114.04178,171.06266,414.71928,282.13458
2591,A_qxRReE,2020-02-20T22:57:01.161Z,U_w8RBv8,2020-02-20T22:57:01.161Z,U_w8RBv8,None,1.2.124.113532.192.70.134.138.20060119.151934....,1.3.46.670589.10.1.1.2163374719.1137684481.588647,1.3.6.1.4.1.9590.100.1.2.316798133213122207700...,L_8GwaX8,1,512,512,"{'x': 300.6775, 'y': 111.07192, 'width': 114.0...",None,[],0,0,Train,G_Zd0jg2,Stomach,bbox,#4fc3f7,,Stomach,[],INSTANCE,34,300.67750,111.07192,114.04178,171.06266,414.71928,282.13458


In [0]:
labels = results['labels']
labels.head()

,groupId,groupName,annotationMode,color,description,labelId,labelName,radlexTagIdsLabel,scope
0,G_MKGlRd,CHAOS,freeform,#4caf50,,L_8W6Ord,Liver,[],INSTANCE
1,G_MKGlRd,CHAOS,freeform,#3f51b5,,L_lPeggd,Right Kidney,[],INSTANCE
2,G_MKGlRd,CHAOS,freeform,#b3e5fc,,L_BnLrqd,Left Kidney,[],INSTANCE
3,G_MKGlRd,CHAOS,freeform,#f0f4c3,,L_8GwaZ8,Spleen,[],INSTANCE
4,G_W5ljNK,Segment,freeform,#e91e63,,L_B7vz28,Liver,[],INSTANCE
